In [1]:
import pandas as pd
import os

In [2]:
PATH_TO_DATASET = r'/media/gagandeep/2E92405C92402AA3/Work/Kaggle/Zips/fake-news'

In [3]:
os.listdir(PATH_TO_DATASET)

['submit.csv', 'test.csv', 'train.csv']

In [4]:
data = pd.read_csv(os.path.join(PATH_TO_DATASET, 'train.csv'))
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# Drop NA values
data = data.dropna()

In [6]:
# Get features and label from the data
X = data.drop('label', axis=1)
y = data['label']
print("Shape of features: {}".format(X.shape))
print("Shape of labels: {}".format(y.shape))

Shape of features: (18285, 4)
Shape of labels: (18285,)


### Our goal is to build a LSTM model - unidirectional and biderectional to classify fake news

In [7]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
# Set the vocabulary size -> the size of our corpus.
voc_size = 5000

In [10]:
# One hot representation
messages = X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gagandeep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# Pre process the data
# The same steps will be used when making predictions
import tqdm
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in tqdm.tqdm(range(0, len(messages))):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)   

100%|██████████| 18285/18285 [00:35<00:00, 510.76it/s]


In [15]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [42]:
import numpy as np
X = np.array(corpus)
y = np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [58]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=voc_size)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)

# Now to make the length of each sentance uniform, we will define a fixed length and pad all the sentances which are less than the ficed length value
fixed_length = 20
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=fixed_length) 
import pickle

po = open('tokenizer.pkl', 'wb')
pickle.dump(tok, po)
po.close()

In [59]:
from tensorflow.keras.layers import Dropout
# Model Creation - Unidirectional
embedding_dim = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_dim, input_length=fixed_length))
model.add(Dropout(0.3))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [61]:
X_test = tok.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, padding='pre', maxlen=fixed_length)

In [41]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
tok.fit_on_texts(corpus)
sequences = tok.texts_to_sequences(corpus)

# Now to make the length of each sentance uniform, we will define a fixed length and pad all the sentances which are less than the ficed length value
fixed_length = 20
embedded_docs = pad_sequences(encoded_corpus, padding='pre', maxlen=fixed_length) 

In [62]:
# Train the model
model.fit(embedded_docs, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/15
12799/12799 [==============================] - 6s 484us/sample - loss: 0.0124 - accuracy: 0.9955 - val_loss: 0.5601 - val_accuracy: 0.9213
Epoch 2/15
12799/12799 [==============================] - 6s 480us/sample - loss: 0.0118 - accuracy: 0.9959 - val_loss: 0.4048 - val_accuracy: 0.9200
Epoch 3/15
12799/12799 [==============================] - 7s 514us/sample - loss: 0.0087 - accuracy: 0.9976 - val_loss: 0.4076 - val_accuracy: 0.9121
Epoch 4/15
12799/12799 [==============================] - 6s 439us/sample - loss: 0.0074 - accuracy: 0.9980 - val_loss: 0.5976 - val_accuracy: 0.9209
Epoch 5/15
12799/12799 [==============================] - 5s 424us/sample - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.5907 - val_accuracy: 0.9203
Epoch 6/15
12799/12799 [==============================] - 6s 481us/sample - loss: 0.0041 - accuracy: 0.9988 - val_loss: 0.6131 - val_accuracy: 0.9214
Epoch 7/15
12799/12799 [===========================

In [63]:
y_pred = model.predict_classes(X_test)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=int32)

In [65]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [67]:
print("-------------Confusion Matrix -----------------")
confusion_matrix(y_test, y_pred)


-------------Confusion Matrix -----------------


array([[2835,  272],
       [ 177, 2202]])

In [68]:
accuracy_score(y_test,y_pred)

0.9181553044112286

In [69]:
print("-------------Classification Report -----------------")
classification_report(y_test, y_pred)

-------------Classification Report -----------------


'              precision    recall  f1-score   support\n\n           0       0.94      0.91      0.93      3107\n           1       0.89      0.93      0.91      2379\n\n    accuracy                           0.92      5486\n   macro avg       0.92      0.92      0.92      5486\nweighted avg       0.92      0.92      0.92      5486\n'

In [70]:
model.save('LSTM_Uni_dropout.h5')

In [71]:
from tensorflow.keras.layers import Bidirectional
# Model Creation - Biidirectional
embedding_dim = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_dim, input_length=fixed_length))
model.add(Bidirectional(LSTM(100, activation='relu')))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
# Train the model
model.fit(embedded_docs, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/30
12799/12799 [==============================] - 16s 1ms/sample - loss: 0.3202 - accuracy: 0.8459 - val_loss: 0.2478 - val_accuracy: 0.9140
Epoch 2/30
12799/12799 [==============================] - 11s 821us/sample - loss: 0.1523 - accuracy: 0.9465 - val_loss: 0.1926 - val_accuracy: 0.9234
Epoch 3/30
12799/12799 [==============================] - 11s 855us/sample - loss: 0.0902 - accuracy: 0.9673 - val_loss: 0.2219 - val_accuracy: 0.9149
Epoch 4/30
12799/12799 [==============================] - 11s 862us/sample - loss: 0.0635 - accuracy: 0.9787 - val_loss: 0.2587 - val_accuracy: 0.9213
Epoch 5/30
12799/12799 [==============================] - 11s 849us/sample - loss: 0.0453 - accuracy: 0.9850 - val_loss: 0.2855 - val_accuracy: 0.9196
Epoch 6/30
12799/12799 [==============================] - 11s 865us/sample - loss: 0.0306 - accuracy: 0.9905 - val_loss: 0.3273 - val_accuracy: 0.9182
Epoch 7/30
12799/12799 [=======================

In [74]:
y_pred = model.predict_classes(X_test)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=int32)

In [75]:
print("-------------Confusion Matrix -----------------")
confusion_matrix(y_test, y_pred)

-------------Confusion Matrix -----------------


array([[2794,  313],
       [ 146, 2233]])

In [79]:
accuracy_score(y_test,y_pred)

0.9163324826831936

In [80]:
print("-------------Classification Report -----------------")
classification_report(y_test, y_pred)

-------------Classification Report -----------------


'              precision    recall  f1-score   support\n\n           0       0.95      0.90      0.92      3107\n           1       0.88      0.94      0.91      2379\n\n    accuracy                           0.92      5486\n   macro avg       0.91      0.92      0.92      5486\nweighted avg       0.92      0.92      0.92      5486\n'

In [81]:
model.save('LSTM_Bi_dropout.h5')